In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1. Load Data
diemrl_df = pd.read_csv("/content/diemrl.csv", encoding="utf-8").drop_duplicates()
diemthu_df = pd.read_csv("/content/diemthu.csv", encoding="utf-8").drop_duplicates()
sinhvien_df = pd.read_csv("/content/sinhvien.csv", encoding="latin1").drop_duplicates()

# 2. Check and Clean Column Names
diemrl_df.columns = diemrl_df.columns.str.strip()
diemthu_df.columns = diemthu_df.columns.str.strip()
sinhvien_df.columns = sinhvien_df.columns.str.strip()

# 3. Data Cleaning
# Ensure 'drl' exists in diemrl_df
if "drl" not in diemrl_df.columns:
    raise KeyError("Cột 'drl' không tồn tại trong diemrl_df. Vui lòng kiểm tra dữ liệu!")
diemrl_df = diemrl_df.dropna(subset=["drl"])

# Ensure necessary columns exist in other DataFrames
required_columns_diemthu = ["mssv", "diem_hp"]
required_columns_sinhvien = ["mssv"]
for col in required_columns_diemthu:
    if col not in diemthu_df.columns:
        raise KeyError(f"Cột '{col}' không tồn tại trong diemthu_df!")
for col in required_columns_sinhvien:
    if col not in sinhvien_df.columns:
        raise KeyError(f"Cột '{col}' không tồn tại trong sinhvien_df!")

# Remove rows with missing values
diemthu_df = diemthu_df.dropna(subset=required_columns_diemthu)
sinhvien_df = sinhvien_df.dropna(subset=required_columns_sinhvien)

# Merge dataframes on 'mssv'
data = pd.merge(diemthu_df, diemrl_df, on="mssv", how="inner")
data = pd.merge(data, sinhvien_df, on="mssv", how="inner")

# Save the cleaned data to a CSV file
output_file_path = "/content/cleaned_data.csv"
data.to_csv(output_file_path, index=False, encoding="utf-8")
output_file_path

# Select features and target
available_features = [col for col in ["drl", "sotc", "namhoc", "hocky"] if col in data.columns]
if len(available_features) < 1:
    raise ValueError("Không có cột nào trong 'features' tồn tại trong dữ liệu!")
target = "diem_hp"
if target not in data.columns:
    raise KeyError(f"Cột '{target}' không tồn tại trong dữ liệu!")

# Drop rows with missing values in selected features/target
data = data.dropna(subset=available_features + [target])

X = data[available_features]
y = data[target]

# 4. Split Data (8:1:1 ratio for Train:Test:Validate)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 5. Build and Train Linear Regression Model
model = LinearRegression()
model.fit(X_train, y_train)

# 6. Evaluate Model
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Add Predicted Scores, Actual Scores, and Error to Test Data
X_test_with_results = X_test.copy()
X_test_with_results["Actual"] = y_test.values  # Điểm thực tế
X_test_with_results["Predicted"] = y_pred_test  # Điểm dự đoán
X_test_with_results["Error"] = X_test_with_results["Actual"] - X_test_with_results["Predicted"]  # Sai số

# Print results
print("\nTest Data with Predictions:")
print(X_test_with_results.head())

# Metrics
metrics = {
    "MAE": mean_absolute_error(y_test, y_pred_test),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred_test)),
    "R-squared": r2_score(y_test, y_pred_test),
}

print("\nModel Performance:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Validate the model on validation set
y_pred_val = model.predict(X_val)
val_metrics = {
    "MAE": mean_absolute_error(y_val, y_pred_val),
    "RMSE": np.sqrt(mean_squared_error(y_val, y_pred_val)),
    "R-squared": r2_score(y_val, y_pred_val),
}

print("\nValidation Performance:")
for metric, value in val_metrics.items():
    print(f"{metric}: {value:.4f}")

# Add Predicted Scores, Actual Scores, and Error to Validation Data
X_val_with_results = X_val.copy()
X_val_with_results["Actual"] = y_val.values  # Điểm thực tế
X_val_with_results["Predicted"] = y_pred_val  # Điểm dự đoán
X_val_with_results["Error"] = X_val_with_results["Actual"] - X_val_with_results["Predicted"]  # Sai số

print("\nValidation Data with Predictions:")
print(X_val_with_results.head())

# 7. Add Custom Accuracy Metric
def calculate_accuracy(y_true, y_pred, tolerance=0.1):
    return np.mean(np.abs(y_true - y_pred) <= tolerance * np.abs(y_true)) * 100

accuracy_test = calculate_accuracy(y_test, y_pred_test, tolerance=0.1)
accuracy_val = calculate_accuracy(y_val, y_pred_val, tolerance=0.1)

print(f"\nAccuracy on Test Set (±10% tolerance): {accuracy_test:.2f}%")
print(f"Accuracy on Validation Set (±10% tolerance): {accuracy_val:.2f}%")


<ipython-input-1-ff72830b41e2>:10: DtypeWarning: Columns (22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68) have mixed types. Specify dtype option on import or set low_memory=False.
  sinhvien_df = pd.read_csv("/content/sinhvien.csv", encoding="latin1").drop_duplicates()



Test Data with Predictions:
        drl  sotc  Actual  Predicted     Error
28418  87.0     3     4.0   6.827268 -2.827268
18294  87.0     4     8.0   6.702712  1.297288
14805  82.0     4     5.0   6.457438 -1.457438
83156  77.0     3     7.3   6.336719  0.963281
38388  94.0     3     6.0   7.170652 -1.170652

Model Performance:
MAE: 1.8100
RMSE: 2.3751
R-squared: 0.0972

Validation Performance:
MAE: 1.7818
RMSE: 2.3307
R-squared: 0.1030

Validation Data with Predictions:
         drl  sotc  Actual  Predicted     Error
73305   95.0     4     5.8   7.095151 -1.295151
64744   63.0     4     6.5   5.525395  0.974605
148612  77.0     4     6.8   6.212163  0.587837
33218   91.0     4     9.5   6.898932  2.601068
138165  98.0     3     0.0   7.366872 -7.366872

Accuracy on Test Set (±10% tolerance): 24.04%
Accuracy on Validation Set (±10% tolerance): 24.04%


preprocessing

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1. Load Data
diemrl_df = pd.read_csv("/content/diemrl.csv", encoding="utf-8").drop_duplicates()
diemthu_df = pd.read_csv("/content/diemthu.csv", encoding="utf-8").drop_duplicates()
sinhvien_df = pd.read_csv("/content/sinhvien.csv", encoding="latin1").drop_duplicates()

# 2. Check and Clean Column Names
diemrl_df.columns = diemrl_df.columns.str.strip()
diemthu_df.columns = diemthu_df.columns.str.strip()
sinhvien_df.columns = sinhvien_df.columns.str.strip()

# 3. Data Cleaning
# Ensure 'drl' exists in diemrl_df
if "drl" not in diemrl_df.columns:
    raise KeyError("Cột 'drl' không tồn tại trong diemrl_df. Vui lòng kiểm tra dữ liệu!")
diemrl_df = diemrl_df.dropna(subset=["drl"])

# Ensure necessary columns exist in other DataFrames
required_columns_diemthu = ["mssv", "diem_hp"]
required_columns_sinhvien = ["mssv"]
for col in required_columns_diemthu:
    if col not in diemthu_df.columns:
        raise KeyError(f"Cột '{col}' không tồn tại trong diemthu_df!")
for col in required_columns_sinhvien:
    if col not in sinhvien_df.columns:
        raise KeyError(f"Cột '{col}' không tồn tại trong sinhvien_df!")

# Remove rows with missing values
diemthu_df = diemthu_df.dropna(subset=required_columns_diemthu)
sinhvien_df = sinhvien_df.dropna(subset=required_columns_sinhvien)

# Merge dataframes on 'mssv'
data = pd.merge(diemthu_df, diemrl_df, on="mssv", how="inner")
data = pd.merge(data, sinhvien_df, on="mssv", how="inner")

# Save the cleaned data to a CSV file
output_file_path = "/content/cleaned_data.csv"
data.to_csv(output_file_path, index=False, encoding="utf-8")
output_file_path